In [26]:
import numpy as np
%matplotlib
import matplotlib.pyplot as plt
from hello.pid.lvpid import PIDController
from hello.pid.delay import m2s, s2m, h2s, seconds, minutes, hours, days, DelayBuffer, DelaySink
from matplotlib.ticker import MultipleLocator, FuncFormatter, FormatStrFormatter
from hello.pid.picker import mk_picker

Using matplotlib backend: TkAgg


In [655]:
from decimal import Decimal as D
class DelaySink():
    def __init__(self, delay, initial, pc_decay=0.95, scalar=1):
        assert 0 < pc_decay <= 1, "don't be a retard"
        self.df1 = self.calc_df(delay, pc_decay)
        if delay:
            self.sink = initial * (1-self.df1) / self.df1
        else:
            self.sink = 0
        self._delay = delay
        self._pc = pc_decay
        self.df2 = scalar*self.df1
        
    def calc_df(self, delay, pc):
        if not delay:
            return 1
        # Attempt to be precise by temporarily using decimal.Decimal
        one = D(1)
        return float(one - (one - D(pc))**(one / D(delay)))
        
    def set_delay(self, delay):
        self.df = self.calc_df(delay, self._pc)
        self._delay = delay
        
    def delay(self, v):
        """ Delay the value v and advance the decay model by 1 second.
        Add v to the current sink, then allow the sink to decay
        according to the decay factor. 
        
        The total decay is given by sink * (df1 + df2). The value
        returned to the system is given by sink * df1. The amount
        lost (e.g., due to radiative heat losses) is given by 
        sink * df2. 
        
        """
        self.sink += v
        d1 = self.sink*self.df1
        d2 = self.sink*self.df2
        self.sink -= d1 + d2
        return d1

In [656]:
class TempProcess():
    def __init__(self, delay, initial=20, output=0, env=18.5, g=0.0019254, k=-0.001579031, g_mult=1):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.tdelay = DelayBuffer(delay, initial).cycle
        self.g = g / 60
        self.k = k / 60
        self.env = env
        self.pv = initial
        
    def step(self, op):
        op = self.tdelay(op)
        dT = self.pv - self.env
        decay = self.k * dT
        gain = self.g * op
        dpv = decay + gain
        self.pv += dpv
        return self.pv
    
class TempProcess2():
    def __init__(self, delay, initial=20, output=0, env=18.5, g=0.0019254, k=-0.001579031, decay=1):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.heatsink = DelaySink(delay, output, 0.95, decay)
        self.g = g / 60
        self.k = k / 60
        self.env = env
        self.pv = initial
        
    def step(self, op):
        op = self.heatsink.delay(op)
        dT = self.pv - self.env
        decay = self.k * dT
        gain = self.g * op
        dpv = decay + gain
        self.pv += dpv
        return self.pv
    
class TempProcess3():
    def __init__(self, delay, initial=20, output=0, env=18.5, g=0.0019254, k=-0.001579031, decay=1):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.heatsink = DelaySink(delay, 0, 0.95, decay)
        self.g = g / 60
        self.k = k / 60
        self.env = env
        self.pv = initial
        
    def step(self, op):
        dT = self.pv - self.env
        decay = self.k * dT
        gain = self.g * op
        gain = self.heatsink.delay(gain)
        dpv = decay + gain
        self.pv += dpv
        return self.pv
    
from math import sqrt
    
class TempProcess4():
    k = -0.0046458109010744375
    
    def __init__(self, delay, initial=20, output=0, env=18.5, g=0.0019254, k=-0.004645810, decay=1):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.heatsink = DelaySink(delay, 0, 0.95, decay)
        self.g = g / 60
        self.k = k / 60
        self.env = env
        self.pv = initial
        
    def step(self, op):
        dT = self.pv - self.env
        decay = self.k * sqrt(dT)a
        gain = self.g * op
        gain = self.heatsink.delay(gain)
        dpv = decay + gain
        self.pv += dpv
        return self.pv

In [657]:
from pysrc.snippets import OptionCategory

offset = 0
p = 60
i = 35
data = []
delay=m2s(15)

class PIDOps(OptionCategory):
    # PID 
    p = 60
    i = 35
    d = 0
    alpha = -1
    linearity = 1
    beta = 1
    gamma = 0
    auto_max = 100
    auto_min = 0
    deadband = 0
    mode = 0
    man_request = 0
    
class TempOps(OptionCategory):    
    
    pid = PIDOps()
    
    # plant    
    k = -0.00357  # C / min * C
    g = 0.002053  # C / min * % 
    k_mult = 1
    g_mult = 1
    delay = m2s(5)
    
    # Simulation
    initial_pv = 20
    env = 20
    initial_output = 0
    setpoint = 37
    end = 5 * hours
    mode = 'o2a'
    process = TempProcess2
    max_iters = 24 * hours
    time_unit = hours
    sink_decay = 0
    

In [658]:
from hello.pid.plots import RingBuffer

class DataQueue():
    """ Standard data container for a single set of data.
    Stores permenant data internally in a RingBuffer.
    For performance reasons, store pending values 
    in a simple python list, and push them to the 
    permanent buffer only when explicitly requested. 
    """
    def __init__(self, pts):
        self._values = RingBuffer(pts)
        self._pending = []
        self.put = self._pending.append
        
    def __len__(self):
        return len(self._values)
        
    def push(self):
        self._values.extend(self._pending)
        self._pending.clear()
        self.put = self._pending.append
        
    def get(self):
        self.push()
        return self._values.get()

    def clear(self):
        self._values.clear()
        self._pending.clear()
        
    def resize(self, new_sz):
        self.push()
        nv = RingBuffer(new_sz)
        nv.extend(self.get())
        self._values = nv

        
def _mk_pid(pidops, pv, sp, req, mode):
    c = PIDController(pgain=pidops.p,               itime=pidops.i,
                      dtime=pidops.d,               auto_max=pidops.auto_max,
                      auto_min=pidops.auto_min,       
                      beta=pidops.beta,
                      linearity=pidops.linearity,   alpha=pidops.alpha,
                      deadband=pidops.deadband,     sp_high=100, sp_low=0,
                      gamma=pidops.gamma,           man_request=pidops.man_request,
                      mode=pidops.mode)
    if mode == "o2a":
        c.off_to_auto(pv, sp)
    elif mode == "m2a":
        c.man_to_auto(pv, sp, req)
    elif mode == "a2a":
        c.man_to_auto(pv, pv, req)
    else:
        raise ValueError(mode)
    return c
        
        
class TempSim():
    def __init__(self, ops, pts=None):
        self.ops = ops
        if pts is None:
            pts = ops.end
        self._mi = self.ops.max_iters
        self._data = {}
        self._pts = pts
        vars = [
            "x", "pv", "env", "hd",
            "uk", "up", "ui", "ud",
        ]
        for v in vars:
            self._add_queue(v)
            
        pv = ops.initial_pv
        sp = ops.setpoint
        hd = ops.initial_output
        
        def minmax(v): return min(100, max(v, 0))
        hd = minmax(hd)
            
        self._pid = _mk_pid(ops.pid, pv, sp, hd, ops.mode)
            
        delay = ops.delay
        ProcessClass = ops.process
    
        self._proc = ProcessClass(delay, pv, hd, ops.env, ops.g*ops.g_mult, 
                                  ops.k*ops.k_mult, ops.sink_decay)
        self._time_unit = ops.time_unit
            
        self._state = s = {}
        s['t'] = 0
        s['sp'] = sp
        s['pv'] = pv
        s['k'] = ops.k
        s['g'] = ops.g
        s['hd'] = hd
        s['env'] = ops.env        
        
    @property
    def state(self):
        return self._state.copy()
        
    def _add_queue(self, name):
        if name in self._data:
            raise ValueError(name)
        self._data[name] = DataQueue(self._pts)
        setattr(self, "_"+name, self._data[name])
        
    def getq(self, name):
        return self._data[name]
    
    def get(self, name):
        return self.getq(name).get()
    
    def legacy_data(self):
        """ Sloppily return data in legacy format 
        so I don't have to refactor a bunch of functions. 
        """
        return self._x.get(), self._pv.get(), self._hd.get()
        
    def sim_iters(self, iters=0):
        
        if iters > self._mi:
            iters = self._mi
        if iters <= 0:
            return
            
        # Optimize bytecode a bit...
        _x = self._x
        _pv = self._pv
        _env = self._env
        _hd = self._hd
        _uk = self._uk
        _up = self._up
        _ui = self._ui
        _ud = self._ud

        s = self._state
        t = s['t']
        sp = s['sp']
        env = s['env']
        pv = s['pv']
        
        time_unit = self._time_unit
        proc = self._proc
        pid = self._pid
        
        while iters > 0:
            iters -= 1
            t += 1
            
            op = pid.step(pv, sp)
            pv = proc.step(op)

            _x.put(t/time_unit)
            _pv.put(pv)
            _env.put(proc.env)
            _hd.put(op)

            _uk.put(pid.Uk)
            _up.put(pid.Up)
            _ui.put(pid.Ui)
            _ud.put(pid.Ud)
            
        s['t'] = t
        s['sp'] = sp
        s['pv'] = pv
        s['hd'] = op
        s['env'] = proc.env
        
    def set_time(self, t):
        self._state['t'] = t
        
    def set_pid_value(self, param, value):
        
        if param == "pgain":
            pid.pgain = value
        elif param == "itime":
            pid.itime = m2s(value)
        elif param == "dtime":
            pid.dtime = m2s(value)
        elif param == "beta":
            pid.b = value
        elif param == "mode":
            pid.set_mode(value, pv, sp)
        elif param == "man":
            pid.man_request = float(value)
        else:
            print("Invalid parameter: %r" % param)
    
    def set_value(self, name, value):
        if name in self._state:
            self._state[name] = value
            if name == "env":
                self._proc.env = value
        else:
            print("Error Invalid Value: %r" % name)

    def clear_data(self):
        for q in self._data.values():
            q.clear()
        self._state['t'] = 0

    def resize(self, n):
        for q in self._data.values():
            q.resize(n)

In [90]:
def declare_list(name):
    if name not in globals():
        globals()[name] = []

# Test 1: Initial PID Exploration for 3L

In [91]:
import sys
_g_axes = []
def mk_axes():
    global _g_axes
    _g_axes = []
    i = 1
    while True:
        try:
            ax = globals()['ax' + str(i)]
        except KeyError:
            break
        else:
            _g_axes.append(ax)
        i += 1

def axes():
    return _g_axes

In [376]:
def setup1(new=False):
    global fig, ax1, ax2, ax3, ax4, ax5, ax6
    
    if not plt.get_fignums() or new:
        fig = plt.figure()
        ax1 = fig.add_subplot(2,1,1)
        ax2 = fig.add_subplot(2,1,2)
        mk_axes()
        for a in ax1, ax2:
            b = a.get_position()
            a.set_position([b.x0, b.y0, b.width*0.8, b.height])
            a.grid()
    else:
        for a in axes():
            a.clear()
            a.grid()
        for t in fig.texts:
            t.remove()
    
    global colors, color
    colors = [
        "blue",
        "red",
        "green",
        "cyan",
        "purple",
        "orange",
        "black"
    ]

    import itertools
    color = itertools.cycle(colors).__next__

In [647]:
def temp_sim1(ops):
    global sim
    sim = TempSim(ops, ops.end)
    sim.sim_iters(ops.end)
    x = sim.get('x')
    pv = sim.get('pv')
    hd = sim.get('hd')
    return x, pv, hd

def _p1(ax, x, y, color, label, ylabel=None):
    ax.plot(x,y, color=color, label=label)
    if ylabel:
        ax.set_ylabel(ylabel)

def plot1(x, pv, hd, label=""):
    c = color()
    _p1(ax1, x, pv, c, label, "Temp (C)")
    _p1(ax2, x, hd, c, label, "Heater Duty (%)")
    
    ax1.axhline(y=ops.setpoint, ls="--", color="black")
    
    ax1.yaxis.set_major_formatter(FormatStrFormatter("%.2f"))
    ax2.yaxis.set_major_formatter(FormatStrFormatter("%d"))
    
    for a in axes():
        a.xaxis.set_major_locator(MultipleLocator(30))
        a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
    
    fig.canvas.flush_events()
    fig.canvas.draw()
    
def run1(ops, label=""):
    global x, pv, hd
    x,pv,hd = temp_sim1(ops)
    plot1(x,pv, hd, label)

def finish1():
    for a in axes():
        if a.legend_:
            mk_picker(fig, a)
    

In [388]:
ops = TempOps()

# PID
ops.pid.p = 40
ops.pid.i = 6
ops.pid.d = 0
ops.pid.alpha = -1
ops.pid.linearity = 1
ops.pid.beta = 0
ops.pid.gamma = 0
ops.pid.auto_max = 50
ops.pid.auto_min = 0
ops.pid.deadband = 0
ops.pid.mode = 0
ops.pid.man_request = 0

# plant    
ops.k = -0.004209176  # C / min * C
ops.g = 0.008794422   # C / min * % 
ops.delay = m2s(5)

# Simulation
ops.initial_pv = 22.9
ops.env = 22.9
ops.initial_output = 0
ops.setpoint = 37
ops.end = 2 * hours
ops.mode = 'o2a'
ops.process = TempProcess
ops.max_iters = 24 * hours
ops.time_unit = minutes

In [118]:
setup1()
for d in (3, 5, 10):
    ops.delay = d*minutes
    run1(ops, "D: %d"%d)
finish1()

# Test 2: Comparison of real vs test data

In [284]:
def setup2(new=False):
    setup1(new)
    _p1(ax1, pvx, pvy, "blue", "Real", "Temp (C)")
    _p1(ax2, hdx, hdy, "blue", "Real", "Heat Duty (%)")

def plot2(x, pv, hd, label=""):
    c = "blue"
    while c == "blue":
        c = color()
    _p1(ax1, x, pv, c, label if label else "Sim")
    _p1(ax2, x, hd, c, label if label else "Sim")
    
    ax1.axhline(y=ops.setpoint, ls="--", color="black")
    ax1.yaxis.set_major_formatter(FormatStrFormatter("%.2f"))
    ax2.yaxis.set_major_formatter(FormatStrFormatter("%d"))
    
    for a in ax1, ax2:
        a.xaxis.set_major_locator(MultipleLocator(30))
        a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        a.set_xlim(0, 120)
        
def run2(ops, label=""):
    global x, pv, hd
    x,pv,hd = temp_sim1(ops)
    plot2(x, pv, hd, label)
    
finish2 = finish1

# Test 3: PID Tuning with 3L Model

In [458]:
def curviness(data):
    # giggity
    g = np.gradient
    d2dt = g(g(data, 1), 1)
    return sum(d2dt)**2

In [877]:
ops = TempOps()

# PID
ops.pid.p = 40
ops.pid.i = 6
ops.pid.d = 0
ops.pid.alpha = -1
ops.pid.linearity = 1
ops.pid.beta = 0
ops.pid.gamma = 0
ops.pid.auto_max = 50
ops.pid.auto_min = 0
ops.pid.deadband = 0
ops.pid.mode = 0
ops.pid.man_request = 0

# plant    
ops.k = -0.004209176  # C / min * C
ops.g = 0.008794422   # C / min * % 
ops.delay = m2s(15)

# Simulation
ops.initial_pv = 22.9
ops.env = 22.9
ops.initial_output = 0
ops.setpoint = 37
ops.end = 2 * hours
ops.mode = 'o2a'
ops.process = TempProcess2
ops.max_iters = 24 * hours
ops.time_unit = minutes
ops.g_mult = 1.2
ops.k_mult = 0.5
ops.sink_decay = 0.2

In [853]:
setup3=setup1
plot3=plot1
def run3(ops, lbl):
    global x,pv,hd
    x,pv,hd = temp_sim1(ops)
    plot3(x,pv,hd,lbl)
def plot3(*args):
    plot1(*args)
    ax1.set_ylim(36, 38)
finish3=finish1    

In [854]:
setup3()
for i in (6, 8, 9):
    ops.pid.p = 40
    ops.pid.i = i
    run3(ops, "i:%d"%i)
finish3()

In [855]:
setup3()
for p in (10, 20, 14):
    ops.pid.p = p
    ops.pid.i = 20
    run3(ops, "P:%d"%p)
finish3()

In [857]:
setup3()
ops.pid.p = 14
ops.pid.i = 20
for env in 16, 20, 24:
    ops.env = env
    run3(ops, "E:%d"%env)
finish3()
ax1.yaxis.set_major_locator(MultipleLocator(0.2))
ops.env=22.9

## Test above simulation in excel

In [863]:
wb2 = xl.Workbooks("2017042116075752.xlsx")
cell_range = wb2.ActiveSheet.Cells.Range
pvx, pvy = get("EE2:EF1497")
hdx, hdy = get("DU2:DV1434")

In [999]:
setup2()
ops.pid.p = 14
ops.pid.i = 20
ops.g_mult = 1.07
ops.k_mult = 0.5
ops.initial_pv = 23.6
ops.delay = 17*minutes
ops.sink_decay = 0.1
ops.env = 18
run2(ops, 'test')
finish2()
ax1.set_ylim(36, 38)
ax1.yaxis.set_major_locator(MultipleLocator(0.2))
#ax1.set_ylim(23, 25)
#ax1.set_xlim(0, 10)

In [1007]:
setup3()
ops.pid.p = 11

for i in 20, 25:
    ops.pid.i = i
    ops.g_mult = 1.3
    ops.k_mult = 0.5
    ops.initial_pv = 23.6
    ops.delay = 20*minutes
    ops.sink_decay = 0.3
    ops.env = 18
    run3(ops, 'I:%d'%i)
finish3()
ax1.set_ylim(36, 38)
ax1.yaxis.set_major_locator(MultipleLocator(0.2))
    #ax1.set_ylim(23, 25)
    #ax1.set_xlim(0, 10)

In [1034]:
setup3()

for p in 10, 11, 12:
    for i in 24, 25, 26:
        ops.pid.p = p
        ops.pid.i = i
        ops.g_mult = 1.3
        ops.k_mult = 0.5
        ops.initial_pv = 23.6
        ops.delay = 20*minutes
        ops.sink_decay = 0.3
        ops.env = 18
        run3(ops, 'P:%d I:%d'%(p, i))
finish3()
ax1.set_ylim(36, 38)
ax1.yaxis.set_major_locator(MultipleLocator(0.2))
    #ax1.set_ylim(23, 25)
    #ax1.set_xlim(0, 10)

In [1041]:
setup2()
ops.pid.p = 11
ops.pid.i = 25
ops.g_mult = 1
ops.k_mult = 0.2
ops.initial_pv = 25.2
ops.env = 25.2
ops.delay = 20*minutes
ops.sink_decay = 0
run2(ops, "Test")
finish2()
ax1.set_ylim(36, 38)
ax1.yaxis.set_major_locator(MultipleLocator(0.2))

# Testing with Excel data for side-by-side comparison

In [1036]:
from officelib.xllib import *
xl = Excel()
wb = xl.ActiveWorkbook
cells = wb.ActiveSheet.Cells
cell_range = cells.Range

In [192]:
def paste():
    x1 = map(float, x)
    pv1 = map(float, pv)
    hd1 = map(float, hd)
    data = list(zip(x1, pv1, hd1))
    c1 = cell_range("EG2")
    c2 = c1.Offset(len(data), len(data[0]))
    cell_range(c1, c2).Value = data

In [402]:
base_g = .397637759 * 1
amax = 50
base_k = -0.004209176

env = 22.9
dt = (37+22)/2 - env
ops.g = (base_g - dt * base_k) / amax
ops.k = base_k
ops.delay = 18 * minutes
ops.process = TempProcess2

setup1()
run1(ops, "test")
run1(ops, "test2")
paste()

In [1037]:
def get(r):
    return [np.array(d) for d in list(zip(*cell_range(r).Value))]
pvx, pvy = get("DY2:DZ998")
hdx, hdy = get("DO2:DP930")

In [466]:
base_g = .397637759 * 1
amax = 50
base_k = -0.004209176

env = 22.9
dt = (37+22)/2 - env
ops.g = (base_g - dt * base_k) / amax
ops.k = base_k
ops.delay = 18 * minutes
ops.process = TempProcess2

setup2()
ops.g_mult = 1.4
_F = 1/2.5
run2(ops)
paste()

In [398]:
setup2()
ops.env=22.9
ops.delay = 15*minutes
ops.process = TempProcess2
for m in 0.2, 0.4, 0.6:
    ops.g_mult = 1 + m
    run2(ops, "M:%.2f"%(m))
finish2()

In [465]:
setup2()
d = 20
ops.delay = 18*minutes
ops.g_mult = 1.5
ops.process = TempProcess2
run2(ops, "P1")
ops.process = TempProcess3
run2(ops, "P2")
finish2()

In [713]:
setup2()
ops.env = 18
ops.delay=15.1*minutes
ops.process = TempProcess2
ops.k_mult = 0.4
x = .1
ops.g_mult = 1 + x
ops.sink_decay = x 
run2(ops, "test")
finish2()
paste()

## Resuming 4/21/17 after a bit of a break

In [714]:
from pysrc.snippets import smooth1
pvx2, pvy2 = smooth1((pvx*60).astype(int), pvy.astype(float))
hdx2, hdy2 = smooth1((hdx*60).astype(int), hdy.astype(float))
pvx2 = np.array([float(f) for f in pvx2])
pvy2 = np.array([float(f) for f in pvy2])
hdx2 = np.array([float(f) for f in hdx2])
hdy2 = np.array([float(f) for f in hdy2])

def rerror(s=None, e=None):
    s = s or 40*60
    e = e or 80*60
    er = pvy2[s:e] - pv[s:e]
    return sum(abs(er))

In [625]:
import peakutils
def slope(x, y):
    s = min(np.where(y == np.max(y))[0])
    e = np.where(y[s:] < 37)[0]
    e = min(e) if e.size else 10000
    return np.polyfit(x[s:s+e], y[s:s+e], 1)

In [525]:
# test impact of env and delay on error
setup2()
ops.k_mult = 0.4
ops.g_mult = 1.3
res = []
for env in range(18, 23, 2):
    for delay in range(16, 19):
        ops.env = env
        ops.delay = delay * minutes
        lbl = "D%d E%d"%(delay, env)
        run2(ops, lbl)
        err = rerror(0, 7200)
        res.append((lbl, err))
finish2()
#paste()
ax1.set_ylim(36.8, 37.8)
ax1.set_xlim(30, 100)
for l,e in sorted(res, key=lambda t: t[1]):
    print(l,e)

D17 E18 105.543299918
D16 E20 144.094834884
D16 E22 205.24524067
D16 E18 248.577207938
D17 E20 261.569868252
D18 E18 321.540571178
D17 E22 467.738264084
D18 E20 524.172192894
D18 E22 721.665755739


In [555]:
# test impact of env and delay on error and slope of PV decay after initial overshoot
setup2()
ops.k_mult = 0.4
ops.g_mult = 1.3
res = []
print(*slope(pvx2/60, pvy2))
for env in 18, 19, 20:
    for delay in (16, 17, 18):
        ops.delay = delay * minutes
        lbl = "D%d E%d"%(delay, env)
        run2(ops, lbl)
        err = rerror(0, 7200)
        res.append((lbl, err, slope(x, pv)))
finish2()
#paste()
ax1.set_ylim(36.8, 37.8)
ax1.set_xlim(30, 100)
for l,e, (m,b) in sorted(res, key=lambda t: t[1]):
    print(l,e, m, b)

-0.0292975987715 39.2418989624
D16 E18 297.560819505 -0.0235603038964 38.9362500279
D16 E19 297.560819505 -0.0235603038964 38.9362500279
D16 E20 297.560819505 -0.0235603038964 38.9362500279
D17 E18 559.968806574 -0.0232732601046 39.0212063526
D17 E19 559.968806574 -0.0232732601046 39.0212063526
D17 E20 559.968806574 -0.0232732601046 39.0212063526
D18 E18 798.206999431 -0.0228498142741 39.0980882472
D18 E19 798.206999431 -0.0228498142741 39.0980882472
D18 E20 798.206999431 -0.0228498142741 39.0980882472


In [636]:
# test impact of k and g multipliers on error and slope of PV decay after initial overshoot
setup2()
ops.k_mult = 0.4
ops.g_mult = 1.3
ops.env = 21
ops.delay = 15*minutes
res = []
print(*slope(pvx2/60, pvy2))
for k_mult in range(1, 11):
    k_mult = k_mult / 10
    ops.k_mult = k_mult
    for g_mult in range(10):
        g_mult = g_mult / 10 + 1
        ops.g_mult = g_mult
        lbl = "k:%.1f g:%.1f"%(k_mult, g_mult)
        x, pv, hd = temp_sim1(ops)
        #run2(ops, lbl)
        err = rerror(0, 7200)
        res.append((k_mult, g_mult, slope(x, pv)))
        print("\r", k_mult, g_mult, slope(x, pv), end="")
finish2()
#paste()
ax1.set_ylim(36.8, 37.8)
ax1.set_xlim(30, 100)

-0.0280962200519 39.1502643993
 1.0 1.9 [ -0.03988626  38.95239163]

(30, 100)

In [637]:
s = slope(pvx2/60, pvy2)[0]
res2 = sorted(res, key = lambda t: abs(s - t[2][0]))
for a, b, (c,d) in res2:
    print(a,b,c,d)
to_test = [(k,g) for k, g, _ in res2[:10]]

0.5 1.6 -0.0280857063078 38.6757169134
0.5 1.5 -0.0286068527984 38.7743067073
0.5 1.7 -0.0275650297996 38.5887684472
0.5 1.4 -0.0291064308504 38.8857343911
0.5 1.8 -0.0270398994326 38.5112161078
0.5 1.3 -0.0296015451012 39.0141178725
0.5 1.9 -0.0265289528344 38.4424773357
0.6 1.9 -0.0298225348056 38.5613714016
0.5 1.2 -0.0300561721514 39.1613269104
0.6 1.8 -0.0303956868874 38.6305369253
0.4 1.0 -0.0257912220159 39.3628766051
0.5 1.1 -0.0305041395884 39.3348592606
0.4 1.1 -0.0254953646513 39.1605240239
0.5 1.0 -0.0309360411084 39.5416382655
0.4 1.2 -0.0252054161633 38.9926959056
0.6 1.7 -0.0310152164614 38.7098674861
0.4 1.3 -0.024902005976 38.8504922288
0.4 1.4 -0.0245837242112 38.7284487941
0.6 1.6 -0.0316779472735 38.8008578461
0.4 1.5 -0.0242548979034 38.6228293893
0.4 1.6 -0.0239113509656 38.5303275261
0.6 1.5 -0.0323260437741 38.9024910619
0.4 1.7 -0.0235419107912 38.4479759154
0.7 1.9 -0.0327706024414 38.6707408356
0.6 1.4 -0.0329727234417 39.017908906
0.4 1.8 -0.0231478213715 38

In [644]:
# test impact of k and g multipliers on error and slope of PV decay after initial overshoot
setup2()
print(slope(pvx2/60, pvy2))
for (km, gm) in to_test:
    ops.k_mult = km
    ops.g_mult = gm
    ops.sink_decay = gm - 1
    lbl = "k:%.1f g:%.1f"%(km, gm)
    run2(ops, lbl)
    print(slope(x, pv), km, gm)
finish2()
#paste()
ax1.set_ylim(36.8, 37.8)
ax1.set_xlim(30, 100)

[ -2.80962201e-02   3.91502644e+01]
[ -3.71320096e-02   4.27935267e+01] 0.5 1.6
[ -3.91251794e-02   4.46930218e+01] 0.5 1.5
[ -3.55460497e-02   4.14864423e+01] 0.5 1.7
[ -4.13344022e-02   4.75769323e+01] 0.5 1.4
[ -3.09202597e-02   4.03447990e+01] 0.5 1.8
[ -4.24056973e-02   5.22294560e+01] 0.5 1.3
[ -2.86550836e-02   3.96222219e+01] 0.5 1.9
[ -3.17420610e-02   3.95962358e+01] 0.6 1.9
[ -3.54003329e-02   6.03992631e+01] 0.5 1.2
[ -3.49652622e-02   4.03415609e+01] 0.6 1.8


(30, 100)

In [668]:
setup2()
ops.k_mult = .5
ops.g_mult = 1
for sm in (0, 0.1, 0.5):
    ops.sink_decay = sm
    lbl = "sm: %.1f"%sm
    run2(ops, lbl)
finish2()
ax1.set_ylim(36.8, 37.8)
ax1.set_xlim(30, 100)

(30, 100)

In [715]:
setup2()
ops.env = 18
ops.delay=15.1*minutes
ops.process = TempProcess2
ops.k_mult = 0.4
x = .1
ops.g_mult = 1 + x
ops.sink_decay = x 
run2(ops, "test")
finish2()
paste()

In [720]:
def rand_float(min, max):
    r = np.random.random()
    rng = max - min
    p = r * rng
    return min + p

In [810]:
setup2()
rf = rand_float
best = None
i = 0
bests = set()
tested = set()
collisions = 0
while True:
    
    env = rf(10, 23)
    g_mult = rf(1, 1.5)
    sm = rand_float(0, 0.6)
    k_mult = rf(0.3, 1)
    delay = rf(10, 20)
    
    key = "%.2f %.2f %.2f %.2f %.2f" % (env, g_mult, sm, k_mult, delay)
    if key in tested:
        collisions += 1
        continue
    tested.add(key)
    i += 1
    
    if not i % 100:
        print("%d tested %d collisions" % (i, collisions))
    
    ops.env = env
    ops.g_mult = g_mult
    ops.sink_decay = sm
    ops.k_mult = k_mult
    ops.delay = delay * minutes
    x, pv, hd = temp_sim1(ops)
    e = err2()
    if best is None or e < best:
        best = e
        bests.add((i, e, env, g_mult, sm, k_mult, delay))
        s = "New Best (%.1f): Env %.1f gm %.2f km %.2f sm %.2f delay = %.2f" % (best, env, g_mult, k_mult, sm, delay)
        s += " %d tested %d bests %d collisions" % (i, len(bests), collisions)
        print(s)
        plot2(x, pv, hd)
        fig.canvas.flush_events()
        fig.canvas.draw()
        fig.canvas.flush_events()
finish2()

New Best (17606.0): Env 11.6 gm 1.35 km 0.61 sm 0.34 delay = 18.78 1 tested 1 bests 0 collisions
New Best (4408.2): Env 21.3 gm 1.26 km 0.47 sm 0.28 delay = 18.14 4 tested 2 bests 0 collisions
100 tested 0 collisions
200 tested 0 collisions
New Best (3988.4): Env 14.2 gm 1.29 km 0.35 sm 0.32 delay = 15.86 220 tested 3 bests 0 collisions
300 tested 0 collisions
New Best (3462.9): Env 10.7 gm 1.30 km 0.36 sm 0.24 delay = 19.69 350 tested 4 bests 0 collisions


KeyboardInterrupt: 

In [812]:
setup2()
def fs(*args):
    return " ".join("%.2f"%a for a in args)
bests2 = sorted(bests_backup, key = lambda k: k[0])
print("  i   g     s  k     d     score")
for i, _, e,g,s,k,d in bests2:
    
    ops.env = e
    ops.g_mult = g
    ops.sink_decay = s
    ops.k_mult = k
    ops.delay = d*minutes
    run2(ops, str(i))
    e = err2()
    print("%4d"%i, fs(g,s,k,d, _))
finish2()
ax1.set_ylim(36.8, 37.8)
ax1.set_xlim(30, 100)

  i   g     s  k     d     score
   1 1.05 0.35 0.74 15.58 43661.41
   2 1.20 0.34 0.58 13.01 16507.37
   4 1.41 0.33 0.43 19.87 5605.33
   8 1.09 0.01 0.43 18.64 4409.39
  12 1.22 0.24 0.37 17.09 4213.56
 122 1.42 0.42 0.33 17.93 3727.14
 385 1.07 0.02 0.47 16.60 1024.56
3037 1.16 0.15 0.67 17.51 705.97


(30, 100)

In [802]:
def err2():
    s = 30*60
    e = 90*60
    s2 = 70*60
    e2 = 120*60
    e1s = sum(abs(pvy2[s:e]-pv[s:e]))
    e2s = sum(abs(hdy2[s:e]-hd[s:e]))
    return np.sqrt(e1s**2 + e2s**2)
    #return e1s, e2s

In [851]:
setup2()
i, _, e,g,s,k,d = bests2[-1]
ops.env = e
ops.g_mult = g
ops.sink_decay = s
ops.k_mult = k
ops.delay = d * minutes
run2(ops, "best")

ops.delay = 16*minutes
ops.g_mult = 1.2
ops.k_mult = .5
ops.sink_decay = 0.25
run2(ops, "test")
paste()
ax1.set_ylim(36.8, 37.8)
ax1.set_xlim(30, 100)

(30, 100)

# 4/24/17 after weekend test

In [1042]:
from officelib.xllib import *
xl = Excel()
wb = xl.ActiveWorkbook
cells = wb.ActiveSheet.Cells
cell_range = cells.Range

In [1060]:
def get(r):
    return [np.array(d) for d in list(zip(*cell_range(r).Value))]
pvx, pvy = get("DZ176:EA404")
hdx, hdy = get("DO175:DP350")
x0 = 180.03  # first "0" entry for temp mode actual
pvx -= x0
hdx -= x0

In [1144]:
setup3()

for p in 10, 11, 12:
    for i in 24, 25, 26:
        ops.pid.p = p
        ops.pid.i = i
        ops.g_mult = 1.3
        ops.k_mult = 0.5
        ops.initial_pv = 25.2
        ops.delay = 20*minutes
        ops.sink_decay = 0.3
        ops.env = 25
        run3(ops, 'P:%d I:%d'%(p, i))
finish3()
ax1.set_ylim(36.5, 37.5)
ax1.yaxis.set_major_locator(MultipleLocator(0.2))
    #ax1.set_ylim(23, 25)
    #ax1.set_xlim(0, 10)

In [1150]:
setup2()
ops.pid.p = 11
ops.pid.i = 25
ops.g_mult = 1.3
ops.k_mult = .5
ops.initial_pv = 25.2
ops.env = 25
ops.delay = 25*minutes
ops.sink_decay = .6
run2(ops, "Test")
finish2()
ax1.set_ylim(36.5, 37.5)
ax1.yaxis.set_major_locator(MultipleLocator(0.2))